In [1]:
from __future__ import print_function, division
import cv2 as cv2
import os
import torch
import torchvision
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, List, Type, Dict, Any
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
import torchvision.models as models
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import time
from tqdm import tqdm
import os.path
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables import Keypoint, KeypointsOnImage
import os
from threading import Thread
from queue import Empty, Queue
import threading
from sklearn.utils import shuffle
import random
import accimage


In [2]:
torch.manual_seed(378) # обнулили генератор рандома у торча
np.random.seed(378)# обнулили генератор рандома у нампая
torch.backends.cudnn.deterministic = True # ждем одинаковый результат у GPU и CPU Для одинаковых данны переданных торчу
torch.backends.cudnn.benchmark = False # вроде аналогично строчке выше
ia.seed(1)

In [3]:
torch.cuda.device_count()
device1 = torch . device ( "cuda:0" if torch . cuda . is_available () else "cpu" )
print(device1)
device2 = torch.device('cpu')

cuda:0


In [4]:
def start_train(model): # запускаем обучение всех слоев
    for param in model.parameters(): 
        param.requires_grad = True
def draw_circle(image, landmarks): # дорисовываем окружность на изображении и возвращаем обратно
    imageWithCircle = cv2.circle(image, (int(landmarks[0][0]),int(landmarks[0][1]) ),int(landmarks[0][2]), (255, 0 , 0),  10)
    return imageWithCircle


def show_landmarks(image, landmarks): # конкретно эта функция, скорее всего не понадобится
    """Show image with landmarks"""
    plt.imshow(draw_circle(image,landmarks) )
    plt.scatter(landmarks[:, 0], landmarks[:, 1], s=10, marker='.', c='r') # начало координат не совпадает , надо учесть
    # для картинки надо, для нейросетки не нужно, начало координат должно совпасть
    # в numpy 0-левая координата - у, первая - иксы(столбцы)
    # порисовать диск солнце порисовать кружочки через opencv(там с координатами тоже не все гладко, надо внимательно смотреть)
    # размеры реальных снимков 1920х1920 а не 1200х1200, надо как-то или к долям 1 или сразу в 1920х1920 формат
    plt.pause(0.001)  # pause a bit so that plots are updated
    
def calculate_loss(model_result : torch.tensor, 
                   data_target  : torch.tensor,
                   loss_function: torch.nn.Module = torch.nn.MSELoss()): #reduction = None
    lossXY =  (loss_function(model_result[:,:2], data_target[:,:2]))**(0.5) # тут из батчей получаю, править
    lossR =  loss_function(model_result[:,2], data_target[:,2]) # потом корень извлечь
    return { lossXY.item(), lossR.item() }



In [5]:
class AddCoords(nn.Module):

    def __init__(self, with_r=False):
        super().__init__()
        self.with_r = with_r

    def forward(self, input_tensor):
        """
        Args:
            input_tensor: shape(batch, channel, x_dim, y_dim)
        """
        batch_size, _, x_dim, y_dim = input_tensor.size()

        xx_channel = torch.arange(x_dim).repeat(1, y_dim, 1)
        yy_channel = torch.arange(y_dim).repeat(1, x_dim, 1).transpose(1, 2)

        xx_channel = xx_channel.float() / (x_dim - 1)
        yy_channel = yy_channel.float() / (y_dim - 1)

        xx_channel = xx_channel * 2 - 1
        yy_channel = yy_channel * 2 - 1

        xx_channel = xx_channel.repeat(batch_size, 1, 1, 1).transpose(2, 3)
        yy_channel = yy_channel.repeat(batch_size, 1, 1, 1).transpose(2, 3)

        ret = torch.cat([
            input_tensor,
            xx_channel.type_as(input_tensor),
            yy_channel.type_as(input_tensor)], dim=1)

        if self.with_r:
            rr = torch.sqrt(torch.pow(xx_channel.type_as(input_tensor) - 0.5, 2) + torch.pow(yy_channel.type_as(input_tensor) - 0.5, 2))
            ret = torch.cat([ret, rr], dim=1)

        return ret


class CoordConv(nn.Module):

    def __init__(self, in_channels, out_channels, with_r=False, **kwargs):
        super().__init__()
        self.addcoords = AddCoords(with_r=with_r)
        in_size = in_channels+2
        if with_r:
            in_size += 1
        self.conv = nn.Conv2d(in_size, out_channels, **kwargs)

    def forward(self, x):
        ret = self.addcoords(x)
        ret = self.conv(ret)
        return ret

In [6]:
class AlexNet(nn.Module):

    def __init__(self, num_classes: int = 1000) -> None:
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            CoordConv(3, 64, kernel_size=11, stride=4, padding=2),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            CoordConv(64, 192, kernel_size=5, padding=2),
            nn.BatchNorm2d(num_features=192),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            CoordConv(192, 384, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=384),
            nn.ReLU(inplace=True),
            CoordConv(384, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(inplace=True),
            CoordConv(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(256 * 6 * 6, 4096),
            nn.BatchNorm1d(num_features=4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1),
            nn.Linear(4096, 4096),
            nn.BatchNorm1d(num_features=4096),
            nn.ReLU(inplace=True),
#             nn.Linear(4096,num_classes),
            nn.Linear(in_features=4096, out_features=256, bias=True),
            nn.BatchNorm1d(num_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=128, bias=True),
            nn.BatchNorm1d(num_features=128),
            nn.ReLU(), # SeLU или GeLU возможно подойдут лучше, надо поиграться
#             nn.Linear(in_features=128, out_features=3, bias=True)
            
            ###### Coords only - without radius ######
            nn.Linear(in_features=128, out_features=2, bias=True)
            ###### Coords only - without radius ######
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


In [7]:
alexnet = AlexNet()

for param in alexnet.parameters(): # разрешаем обучаться alexnet
    param.requires_grad = True

alexnet = alexnet.to(device1)


# упомянуть челиков из imgaug
## тут агментация данных дальше


In [9]:
seq = iaa.Sequential([
    iaa.Fliplr(0.5),# horizontally flip 20% of the images flipud
    iaa.Flipud(0.5),# vertically flip 20% of the images flipud добавить на 180 градусов поворот
    iaa.GaussianBlur(sigma=(0, 5)), # blur images with a sigma of 0 to 0.5
    #iaa.Invert(0.1) # пока уберем
    iaa.Affine(rotate=(-180,180))# поворот на 180 через афииные преобразования
])


In [10]:
def find_files(dataset_dir):
    image_filenames = [join(dataset_dir, x) for x in listdir(dataset_dir)]
    return image_filenames


class thread_killer(object):    
    """Boolean object for signaling a worker thread to terminate"""
    def __init__(self):
        self.to_kill = False

    def __call__(self):
        return self.to_kill

    def set_tokill(self, tokill):
        self.to_kill = tokill


def threaded_batches_feeder(tokill, batches_queue, dataset_generator):
    while tokill() == False:
        for img, landmarks in dataset_generator:
            batches_queue.put((img, landmarks), block=True)
            if tokill() == True:
                return


def threaded_cuda_batches(tokill,cuda_batches_queue,batches_queue):
    while tokill() == False:
        (img, landmarks) = batches_queue.get(block=True)
        img = torch.from_numpy(img)
        landmarks = torch.from_numpy(landmarks)       
        
        img = Variable(img.float()).to(device1)
        
#         landmarks = Variable(landmarks.float()).view(-1,3).to(device1)
        ###### Coords only - without radius ######
        landmarks = Variable(landmarks.float()).view(-1,2).to(device1)
        ###### Coords only - without radius ######
        
        cuda_batches_queue.put((img, landmarks), block=True)
        
        if tokill() == True:
            return


class threadsafe_iter:
    """
    Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        with self.lock:
            return next(self.it)

def get_objects_i(objects_count):
    """Cyclic generator of paths indices
    """
    current_objects_id = 0
    while True:
        yield current_objects_id
        current_objects_id  = (current_objects_id + 1) % objects_count


In [11]:
class SunLandmarksDataset(Dataset):
    """Sun Landmarks dataset."""

    def __init__(self, csv_file, root_dir, batch_size, transform = None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.landmarks_frame = pd.read_csv(csv_file,delimiter=',')
        self.root_dir = root_dir
        self.batch_size = batch_size
        
        self.objects_id_generator = threadsafe_iter(get_objects_i(self.landmarks_frame.shape[0]))
        
        self.lock = threading.Lock()  # mutex for input path
        self.yield_lock = threading.Lock()  # mutex for generator yielding of batch
        self.init_count = 0
        self.cache = {}

    def __len__(self):
        return len(self.landmarks_frame)

    def shuffle(self):
        self.landmarks_frame = shuffle(self.landmarks_frame)
    
    def __iter__(self):
        while True:
            with self.lock:
                if (self.init_count == 0):
                    self.shuffle()
                    self.imgs = []
                    self.landmarks = []
                    self.init_count = 1
            
            for obj_id in self.objects_id_generator:
                img_name = os.path.join(self.root_dir , self.landmarks_frame.iloc[obj_id, 0])
                if img_name in self.cache:
                    img = self.cache[img_name]
                else:
                    img = accimage.Image(img_name)
                    image_np = np.empty([img.channels, img.height, img.width], dtype=np.uint8)
                    img.copyto(image_np)
                    self.cache[img_name] = img
                    
                img = np.transpose(image_np, (1, 2, 0))
                
                landmarks = self.landmarks_frame.iloc[obj_id, 1:]
                landmarks = np.array([landmarks])
                landmarks = landmarks.astype('float').reshape(1, 3)
                
                kps = KeypointsOnImage([
                        Keypoint(x=landmarks[0,0]*1920, y=landmarks[0,1]*1920) # домножил, тк нормировал
                ], shape=img.shape)
                image_aug, landmarks_after =seq(image = img, keypoints = kps) # тут применяю аугментацию к фотке и целевой переменной
                landmarks[0,0] = landmarks_after.keypoints[0].x/1920 # обратно отнормировал 
                landmarks[0,1] = landmarks_after.keypoints[0].y/1920
                img = image_aug
                
                img = img.transpose((2, 0, 1))[np.newaxis,...] # для тензоров
                landmarks = landmarks[np.newaxis,...]
                
                ###### Coords only - without radius ######
                landmarks = landmarks[:,:,:2]
                ###### Coords only - without radius ######
                
                # Concurrent access by multiple threads to the lists below
                with self.yield_lock:
                    if (len(self.imgs)) < self.batch_size:
                        self.imgs.append(img)
                        self.landmarks.append(landmarks)
                    if len(self.imgs) % self.batch_size == 0:
                        self.imgs = np.concatenate(self.imgs, axis=0)
                        self.landmarks = np.concatenate(self.landmarks, axis=0)
                        yield (self.imgs, self.landmarks)
                        self.imgs, self.landmarks = [], []
                        
            # At the end of an epoch we re-init data-structures
            with self.lock:
                self.landmarks_frame = shuffle(self.landmarks_frame)
                self.init_count = 0


In [12]:
batch_size = 8
sun_dataset_train = SunLandmarksDataset(csv_file='sun_disk_pos_database01train.csv',
                                        root_dir='/app/images',
                                        transform=None,
                                        batch_size=batch_size)
sun_dataset_test = SunLandmarksDataset(csv_file='sun_disk_pos_database01test.csv',
                                       root_dir='/app/images',
                                       transform=None,
                                       batch_size=batch_size)

In [16]:
def train_model(model: torch.nn.Module, 
                train_dataset: torch.utils.data.Dataset,
                val_dataset: torch.utils.data.Dataset,
                loss_function: torch.nn.Module = torch.nn.MSELoss(), # loss_function: torch.nn.Module = torch.nn.CrossEntropyLoss() 
                optimizer_class: Type[torch.optim.Optimizer] = torch.optim,
                optimizer_params: Dict = {},
                lr_scheduler_class: Any = torch.optim.lr_scheduler.CosineAnnealingLR,
                initial_lr = float,
                lr_scheduler_params: Dict = {},
                max_epochs = 200):

    optimizer = torch.optim.Adam(model.parameters(), lr=initial_lr, **optimizer_params)
    lr_scheduler = lr_scheduler_class(optimizer, **lr_scheduler_params)

    best_val_loss = None
    best_epoch = None
    loss_value = []
    
    batches_queue_length =16

    train_batches_queue = Queue(maxsize=batches_queue_length)
    train_cuda_batches_queue = Queue(maxsize=8)
    train_thread_killer = thread_killer()
    train_thread_killer.set_tokill(False)
    preprocess_workers = 16

    for _ in range(preprocess_workers):
        thr = Thread(target=threaded_batches_feeder, args=(train_thread_killer, train_batches_queue, train_dataset))
        thr.start()

    train_cuda_transfers_thread_killer = thread_killer()
    train_cuda_transfers_thread_killer.set_tokill(False)
    train_cudathread = Thread(target=threaded_cuda_batches, args=(train_cuda_transfers_thread_killer, train_cuda_batches_queue,
                                                                  train_batches_queue))
    train_cudathread.start()
    

    test_batches_queue = Queue(maxsize=batches_queue_length)
    test_cuda_batches_queue = Queue(maxsize=8)
    test_thread_killer = thread_killer()
    test_thread_killer.set_tokill(False)
    preprocess_workers = 16

    for _ in range(preprocess_workers):
        thr = Thread(target=threaded_batches_feeder, args=(train_thread_killer, test_batches_queue, sun_dataset_test))
        thr.start()

    test_cuda_transfers_thread_killer = thread_killer()
    test_cuda_transfers_thread_killer.set_tokill(False)
    test_cudathread = Thread(target=threaded_cuda_batches, args=(test_cuda_transfers_thread_killer, test_cuda_batches_queue,
                                                                  test_batches_queue))
    test_cudathread.start()
    
    
    Steps_Per_Epoch_Train = len(train_dataset)/batch_size # количество итераций обучения и валидации
    Steps_Per_Epoch_Test = len(val_dataset)/batch_size
    
    
    for epoch in range(max_epochs):
        clear_output() # слишком много загрузчиков
        print(f'Epoch {epoch}')
        train_single_epoch(model, optimizer, loss_function, train_cuda_batches_queue,Steps_Per_Epoch_Train) # передать х
        val_metrics = validate_single_epoch(model, loss_function, test_cuda_batches_queue,Steps_Per_Epoch_Test)
        loss_value = np.append(loss_value,val_metrics) # добавляю в массив значение функци потерь
        np.save('loss_value_alexnetXY',loss_value )
        print(f'Validation metrics: \n{val_metrics}')
        


        lr_scheduler.step() #lr_scheduler.step(val_metrics['loss'])


        
        if best_val_loss is None or best_val_loss > val_metrics:
            print(f'Best model yet, saving')
            best_val_loss = val_metrics
            best_epoch = epoch
            torch.save(model, 'saved_models/best_model_alexnetXY.pth') 

        
#         if epoch - best_epoch > early_stopping_patience:
#             print('Early stopping triggered')
#             return
        

    train_thread_killer.set_tokill(True) # убиваю потокои, так же убить валидационные
    train_cuda_transfers_thread_killer.set_tokill(True)
    for _ in range(preprocess_workers):
        try:
            # Enforcing thread shutdown
            train_batches_queue.get(block=True, timeout=1)
            train_cuda_batches_queue.get(block=True, timeout=1)
        except Empty:
            pass
        
        
    test_thread_killer.set_tokill(True) # убиваю потокои, так же убить валидационные
    test_cuda_transfers_thread_killer.set_tokill(True)
    for _ in range(preprocess_workers):
        try:
            # Enforcing thread shutdown
            test_batches_queue.get(block=True, timeout=1)
            test_cuda_batches_queue.get(block=True, timeout=1)
        except Empty:
            pass


In [14]:
def validate_single_epoch(model: torch.nn.Module,
                        loss_function: torch.nn.Module, 
                        cuda_batches_queue: Queue, 
                        Per_Step_Epoch:int):
    



    test_loss = 0
    correct = 0
    model.eval()
    pbar = tqdm(total=Per_Step_Epoch)
    for batch_idx in range(int(Per_Step_Epoch)): # тут продумать 
        data_image, target = cuda_batches_queue.get(block=True)
        data_out = model(data_image)
        loss = loss_function(data_out, target) # посчитали на изображении
        test_loss += loss.item() # прибавили
        pbar.update(1)
        pbar.set_postfix({'loss': loss.item()})
    test_loss /= Per_Step_Epoch*batch_size

    return {test_loss}


In [15]:
def train_single_epoch(model: torch.nn.Module,
                    optimizer: torch.optim.Optimizer, 
                    loss_function: torch.nn.Module,
                    cuda_batches_queue: Queue, 
                    Per_Step_Epoch:int):
                    #data_loader: torch.utils.data.DataLoader):

    model.train()
    pbar = tqdm(total=Per_Step_Epoch)
    for batch_idx in range(int(Per_Step_Epoch)): # тут продумать 
        data_image, target = cuda_batches_queue.get(block=True)
        target = Variable(target).view(-1,2)
        target = target.to(device1)
        optimizer.zero_grad() # обнулили\перезапустии градиенты для обратного распространения
        data_out = model(data_image)# применили модель к данным
        loss = loss_function(data_out, target) # применили фуннкцию потерь
        loss.backward() # пошли по графу нейросетки обратно
        optimizer.step()# выполняем наш градиентный спуск по вычисленным шагам в предыдущей строчке
        pbar.update(1)
        pbar.set_postfix({'loss': loss.item()})
        
        
        
    return {'loss': loss.item()}

In [ ]:

    train_model(alexnet, 
            train_dataset = sun_dataset_train, 
            val_dataset =  sun_dataset_test,  
            loss_function = torch.nn.MSELoss(), 
            lr_scheduler_class = torch.optim.lr_scheduler.CosineAnnealingLR,
            lr_scheduler_params = {'T_max': 100, 'last_epoch': -1, 'eta_min': 1e-7},
            initial_lr=1e-4)